In [1]:
import time
import pandas as pd
import numpy as np
import warnings
from scipy.stats import pearsonr, norm
from scipy.optimize import minimize, fmin_l_bfgs_b, shgo
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C

## Inputs

In [3]:
fname="../wall/gpr/generation_2_rw.xlsx" # SPECIFY FILE TO BE READ..COLUMNS: 'lab' (label), 'a_i' (design paramters), 'Frac'(Yf), 'stan' (std err of Yf), 'k80' (K), 'k80_err' (K error)
gen = pd.read_excel(fname,index_col='lab')
n_des_param = 4 # number of design paramters
opt_global = False # Flag to choose between global and local optimization for Expected Improvement
local_method = "nelder-mead" # or "BFGS" or "L-BFGS-B"
nmot = 27.552 #Y^f NMO
nmok = -12.452 #K NMO
b = [[5.,6.5],[1,100],[1,5],[1.8,1.83]] # Bounds over design parameters
bounds = np.array(b)
gen

,a0,a1,a2,a3,N*,Frac,err,stan,k80,k80_err
lab,,,,,,,,,,
nmo,5.89000,17.90000,2.49000,1.82300,105.8,27.552083,44.34410,3.651754,-12.45200,0.1188
P2,5.77569,24.25110,3.57752,1.82108,156.5,40.755208,93.28360,7.681942,-5.44875,0.0722
P8,5.50000,15.49900,3.33500,1.82300,339.9,88.515625,41.28610,3.399927,-5.79103,0.0637
nP2,5.51190,15.74929,3.33840,1.82430,358.9,93.463542,18.83530,1.551094,-5.54803,0.0624
nP6,5.51140,15.77750,3.34330,1.82560,301.9,78.619792,56.07820,4.618062,-5.32668,0.0327
P9,5.51500,15.66000,3.34560,1.82560,287.0,74.739583,72.55500,5.974933,-5.29832,0.0000
r1,5.50000,11.69980,5.00000,1.82400,13.8,3.593750,8.56089,0.704993,-20.00000,0.0000
r2,5.51170,15.58190,5.00000,1.82430,16.3,4.244792,7.58727,0.624815,-20.00000,0.0000
r3,5.50000,14.64490,5.00000,1.82400,12.5,3.255208,4.76678,0.392546,-20.00000,0.0000


# GPR Thermodynamic - Kernel Convergence

In [4]:
start = time.time()
niter = 20 # number of outer optimizations over kernel parameters; need to vary this based on dataset
kernel_nu = 0.5 # need to vary between 0.5,1.5,2.5 based on data

#** function to calculate averaged correlation between training and test datasets **
def obj(k): 
    corr_tr=0
    corr_tt=0
    samp_iter=0
    while samp_iter < 100:
        samp_iter = samp_iter + 1
        sample = gen.sample(frac=0.5)
        xtrain = sample[['a0','a1','a2','a3']].to_numpy() #need to specify column names of design parameters
        xtrain.reshape(-1,n_des_param)
        ytrain = sample['Frac'].to_numpy()
        ytrain_err = sample['stan'].to_numpy()
        ytrain = ytrain.ravel()
        kernel = C(k[0],(1e-2,1e2)) * Matern((k[1:]), length_scale_bounds=(1e-3,1e3),nu=kernel_nu)
        warnings.filterwarnings("ignore")
        gp_therm = GaussianProcessRegressor(kernel=kernel, alpha=ytrain_err, n_restarts_optimizer=5)
        gp_therm.fit(xtrain,ytrain)
        xtest=[[0]*n_des_param]
        sim=[0]
        sim_err = [0]
        for index,row in gen.iterrows():
            if index in sample.index:
                continue
            else:
                r=row[['a0','a1','a2','a3']].to_numpy() # again need to specify column names
                xtest = np.vstack([xtest,r])
                sim.append(row['Frac'])
                sim_err.append(row['stan'])
        y_pred, sigma = gp_therm.predict(xtest, return_std=True)
        y_pred_cont, sig_cont = gp_therm.predict(xtrain, return_std=True)
        sigma = [s/2 for s in sigma]
        cptr,_ = pearsonr(y_pred_cont,ytrain)
        cptt,_ = pearsonr(y_pred,sim)
        corr_tr = corr_tr + cptr
        corr_tt = corr_tt + cptt
    corr_tr = corr_tr/samp_iter
    corr_tt = corr_tt/samp_iter
    return -corr_tt

k = np.array([1]*(n_des_param+1)) #initial kernel parameters
result = minimize(obj,k,method='nelder-mead',options={"maxiter":niter})
solution = result['x'] #optimized kernel parameters
evaluation = -obj(solution)
print("Converged kernel params: ", solution)
print("Converged test data pearson correlation = ", evaluation)

xtr = gen[['a0','a1','a2','a3']].to_numpy()
ytr = gen['Frac'].to_numpy()
ytr_err = gen['stan'].to_numpy()
ytr = ytr.ravel()
conv_kernel = C(solution[0],(1e-2,1e2)) * Matern((solution[1:]), length_scale_bounds=(1e-3,1e3),nu=kernel_nu)
gpr_therm = GaussianProcessRegressor(kernel=conv_kernel,alpha=ytr_err,n_restarts_optimizer=10)
gpr_therm.fit(xtr,ytr)
print("Total run time = {} seconds".format(time.time()-start))

Converged kernel params:  [0.9692 0.972  1.0392 1.0392 1.0392]
Converged test data pearson correlation =  0.9020391135500766
Total run time = 479.02845311164856 seconds


# GPR Kinetic - Kernel Convergence

In [5]:
start = time.time()
niter = 20 #number of outer optimizations over kernel parameters; need to vary this based on dataset
kernel_nu = 0.5 # need to vary between 0.5,1.5,2.5 based on data

#** function to calculate averaged correlation between training and test datasets **
def obj(k):
    corr_tr=0
    corr_tt=0
    samp_iter=0
    while samp_iter < 100:
        samp_iter = samp_iter + 1
        sample = gen.sample(frac=0.5)
        xtrain = sample[['a0','a1','a2','a3']].to_numpy() #need to specify column names of design parameters
        xtrain.reshape(-1,n_des_param)
        ytrain = sample['k80'].to_numpy()
        ytrain_err = sample['k80_err'].to_numpy()
        ytrain = ytrain.ravel()
        kernel = C(k[0],(1e-2,1e2)) * Matern((k[1:]), length_scale_bounds=(1e-3,1e3),nu=kernel_nu)
        warnings.filterwarnings("ignore")
        gp_kin = GaussianProcessRegressor(kernel=kernel, alpha=ytrain_err, n_restarts_optimizer=5)
        gp_kin.fit(xtrain,ytrain)
        xtest=[[0]*n_des_param]
        sim=[0]
        sim_err = [0]
        for index,row in gen.iterrows():
            if index in sample.index:
                continue
            else:
                r=row[['a0','a1','a2','a3']].to_numpy() # again need to specify column names
                xtest = np.vstack([xtest,r])
                sim.append(row['k80'])
                sim_err.append(row['k80_err'])
        y_pred, sigma = gp_kin.predict(xtest, return_std=True)
        y_pred_cont, sig_cont = gp_kin.predict(xtrain, return_std=True)
        sigma = [s/2 for s in sigma]
        cptr,_ = pearsonr(y_pred_cont,ytrain)
        cptt,_ = pearsonr(y_pred,sim)
        corr_tr = corr_tr + cptr
        corr_tt = corr_tt + cptt
    corr_tr = corr_tr/samp_iter
    corr_tt = corr_tt/samp_iter
    return -corr_tt

k = np.array([1]*(n_des_param+1)) # initial kernel paramteres
result = minimize(obj,k,method='nelder-mead',options={"maxiter":niter})
solution = result['x'] #optimized kernel parameters
evaluation = -obj(solution)
print("Converged kernel params: ", solution)
print("Converged test data pearson correlation = ", evaluation)

xtr = gen[['a0','a1','a2','a3']].to_numpy()
ytr = gen['k80'].to_numpy()
ytr_err = gen['k80_err'].to_numpy()
ytr = ytr.ravel()
conv_kernel = C(solution[0],(1e-2,1e2)) * Matern((solution[1:]), length_scale_bounds=(1e-3,1e3),nu=kernel_nu)
gpr_kin = GaussianProcessRegressor(kernel=conv_kernel,alpha=ytr_err,n_restarts_optimizer=10)
gpr_kin.fit(xtr,ytr)
print("Total run time = {} seconds".format(time.time()-start))

Converged kernel params:  [1.05340271 0.99791069 0.97187546 1.04164243 0.98382888]
Converged test data pearson correlation =  0.7472860672999964
Total run time = 470.00868487358093 seconds


# Maximize Expected Improvement, Propose design vectors

In [11]:
ll = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] #lambda 
xkr = gen[['a0','a1','a2','a3']].to_numpy()
yk = gen['k80'].to_numpy()
yt = gen['Frac'].to_numpy()

def func(x,l):
    gt, sigt = gpr_therm.predict(x, return_std=True)
    gk, sigk = gpr_kin.predict(x, return_std=True)
    mean = l*gt + (1-l)*gk
    sigma = l*sigt + (1-l)*sigk
    return mean, sigma

def expI(x, l, f, evaluated_loss, n_params=n_des_param):
    x_to_predict = x.reshape(-1,n_params)
    mu, sigma = f(x_to_predict,l)
    loss_optimum = np.max(evaluated_loss)
    with np.errstate(divide='ignore'):
        Z = (mu-loss_optimum)/sigma
        expected_improvement = (mu-loss_optimum) * norm.cdf(Z) + sigma*norm.pdf(Z)
        expected_improvement[sigma == 0.0] == 0.0
    return expected_improvement

count=0
ei=0
for i in range(len(ll)):
    ev_loss = ll[i]*gpr_therm.predict(xkr) + (1-ll[i])*gpr_kin.predict(xkr)
    def f_to_call(x):
        res = expI(x,ll[i],func,ev_loss,n_params=n_des_param)
        return -res
    for k in range(1): # Vary this to control number of predicted potentials for each \lambda
        x_random = np.random.uniform(bounds[:,0],bounds[:,1], size=(1,n_des_param))
        xr = [[x_random]]
        if (opt_global==True):
            result = shgo(f_to_call,bounds)
        else:
            result = minimize(f_to_call,xr,method=local_method,bounds=bounds)
        gt,st = gpr_therm.predict([result['x']],return_std=True)
        gk,sk = gpr_kin.predict([result['x']],return_std=True)
        if ((gt>nmot and gk>nmok)or ((st>2 and st<6) or (sk>2 and sk<6))):
            exp_imp = -f_to_call(result['x'])
            print("lambda = {}: ({}) des. param. = {}, Yf_pred = {}, sigmaY = {}, K_pred = {}, sigmaK = {}, EI = {}".format(ll[i],count+1,result['x'],gt,st,gk,sk,exp_imp))
            ei = ei + exp_imp
            count+=1

avg_ei = ei/count
print("Total number of predicted potentials = {}".format(count))
print("Average Expected Improvement = {}".format(avg_ei))

lambda = 0: (1) des. param. = [ 6.5        95.94170941  3.43912993  1.8       ], Yf_pred = [43.52328371], sigmaY = [8.49703667], K_pred = [-6.46688172], sigmaK = [2.73981999], EI = [0.60668622]
lambda = 0.1: (2) des. param. = [  5.         100.           1.99077749   1.8       ], Yf_pred = [0.48746336], sigmaY = [9.99837879], K_pred = [-14.3880469], sigmaK = [3.81951083], EI = [6.76083375e-05]
lambda = 0.2: (3) des. param. = [ 5.00023588 45.27424204  3.33629905  1.81898669], Yf_pred = [86.9531505], sigmaY = [2.43551256], K_pred = [-5.88017444], sigmaK = [1.78717066], EI = [0.35920978]
lambda = 0.3: (4) des. param. = [ 6.10233615 30.0329452   3.33729786  1.83      ], Yf_pred = [88.80177241], sigmaY = [1.84665448], K_pred = [-5.67286011], sigmaK = [1.26861398], EI = [0.29971572]
lambda = 0.4: (5) des. param. = [ 6.5        22.34722852  3.33780129  1.8       ], Yf_pred = [89.81210532], sigmaY = [1.51331924], K_pred = [-5.58277144], sigmaK = [0.86944997], EI = [0.27659321]
lambda = 0.5: (6